<a href="https://colab.research.google.com/github/Martipetti/MachineLearning/blob/develop-martipetti/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network

In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense

In [4]:
link = 'http://tennis-data.co.uk/2023/2023.xlsx'
totalData = pd.read_excel(link)

In [5]:
totalData

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,Lsets,Comment,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,Adelaide International 1,2023-01-01,ATP250,Outdoor,Hard,1st Round,3,Giron M.,...,1.0,Completed,1.91,1.91,1.93,1.95,1.99,1.95,1.89,1.89
1,1,Adelaide,Adelaide International 1,2023-01-01,ATP250,Outdoor,Hard,1st Round,3,Mcdonald M.,...,0.0,Retired,1.36,3.20,1.39,3.25,1.44,3.40,1.36,3.12
2,1,Adelaide,Adelaide International 1,2023-01-02,ATP250,Outdoor,Hard,1st Round,3,Kecmanovic M.,...,0.0,Completed,1.57,2.38,1.58,2.53,1.64,2.53,1.58,2.36
3,1,Adelaide,Adelaide International 1,2023-01-02,ATP250,Outdoor,Hard,1st Round,3,Nishioka Y.,...,1.0,Completed,3.75,1.29,4.00,1.28,4.00,1.31,3.56,1.29
4,1,Adelaide,Adelaide International 1,2023-01-02,ATP250,Outdoor,Hard,1st Round,3,Popyrin A.,...,0.0,Completed,6.50,1.11,6.20,1.15,6.75,1.18,6.04,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3,Alcaraz C.,...,0.0,Completed,1.67,2.20,1.78,2.16,1.78,2.30,1.68,2.19
2699,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,...,0.0,Completed,1.62,2.30,1.74,2.23,1.76,2.35,1.66,2.22
2700,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3,Sinner J.,...,1.0,Completed,1.67,2.20,1.69,2.31,1.71,2.36,1.64,2.26
2701,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3,Djokovic N.,...,0.0,Completed,1.62,2.30,1.67,2.36,1.70,2.43,1.63,2.28


Cleaning DataSet

In [ ]:
totalData = totalData.drop(["Date", ])